In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession \
    .builder \
    .appName("pyspark-notebook") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [2]:
data = spark.read.options(header='True', inferSchema='True', delimiter=',') \
    .csv("/opt/workspace/FakeNameGenerator.com_000ba228.csv")

data.printSchema()
data.show(n=5)

root
 |-- Number: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- NameSet: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- GivenName: string (nullable = true)
 |-- MiddleInitial: string (nullable = true)
 |-- Surname: string (nullable = true)
 |-- StreetAddress: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- StateFull: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- CountryFull: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- Password: string (nullable = true)
 |-- BrowserUserAgent: string (nullable = true)
 |-- TelephoneNumber: string (nullable = true)
 |-- TelephoneCountryCode: integer (nullable = true)
 |-- MothersMaiden: string (nullable = true)
 |-- Birthday: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- TropicalZodiac: string (nullable = tru

In [11]:
data.write.format("delta").mode("overwrite").save("/opt/workspace/Fakenames-delta-table")

In [17]:
delta_read = spark.read.format("delta").load("/opt/workspace/Fakenames-delta-table")
delta_read.show(n=1)

+------+------+----------+-----+---------+-------------+-------+---------------+--------------+-----+----------+-------+-------+-----------+--------------------+--------+---------+--------------------+---------------+--------------------+-------------+---------+---+--------------+------+----------------+----+---------+----------+--------------------+----------------+-------------+-----+--------------------+-------+-----------------+-----------------+---------+------+---------+----------+-----------+--------------------+----------+----------+
|Number|Gender|   NameSet|Title|GivenName|MiddleInitial|Surname|  StreetAddress|          City|State| StateFull|ZipCode|Country|CountryFull|        EmailAddress|Username| Password|    BrowserUserAgent|TelephoneNumber|TelephoneCountryCode|MothersMaiden| Birthday|Age|TropicalZodiac|CCType|        CCNumber|CVV2|CCExpires|NationalID|                 UPS|WesternUnionMTCN|MoneyGramMTCN|Color|          Occupation|Company|          Vehicle|           Domai

In [22]:
delta_read.createOrReplaceTempView("TBLFAKENAMES")

In [24]:
delta_read.describe().show()

+-------+-----------------+------+----------+------+---------+-------------+---------+--------------+-----------+------+--------------------+------------------+-------+-----------+--------------------+--------+-----------+--------------------+---------------+--------------------+-------------+--------+------------------+--------------+----------+--------------------+-----------------+---------+----------+--------------------+-------------------+-------------------+------+------------+-----------------+------------------+----------------+---------+------------------+-----------------+----------+-----------------+--------------------+------------------+------------------+
|summary|           Number|Gender|   NameSet| Title|GivenName|MiddleInitial|  Surname| StreetAddress|       City| State|           StateFull|           ZipCode|Country|CountryFull|        EmailAddress|Username|   Password|    BrowserUserAgent|TelephoneNumber|TelephoneCountryCode|MothersMaiden|Birthday|               Age

In [21]:
spark.sql("SELECT Number TBLFAKENAMES WHERE Number < 2").show()

AnalysisException: cannot resolve '`Number`' given input columns: []; line 1 pos 33;
'Project ['Number AS TBLFAKENAMES#4365]
+- 'Filter ('Number < 2)
   +- OneRowRelation
